##### Copyright 2018 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [0]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

# Sovradattamento e sottoadattamento

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/keras/overfit_and_underfit"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />Visualizza su TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/keras/overfit_and_underfit.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Esegui in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/keras/overfit_and_underfit.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />Visualizza il sorgente su GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/keras/overfit_and_underfit.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Scarica notebook</a>
  </td>
</table>

Note: La nostra comunità di Tensorflow ha tradotto questi documenti. Poichè queste traduzioni sono *best-effort*, non è garantito che rispecchino in maniera precisa e aggiornata la [documentazione ufficiale in inglese](https://www.tensorflow.org/?hl=en). 
Se avete suggerimenti per migliorare questa traduzione, mandate per favore una pull request al repository Github [tensorflow/docs](https://github.com/tensorflow/docs). 
Per proporsi come volontari alla scrittura o alla review delle traduzioni della comunità contattate la 
[mailing list docs@tensorflow.org](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs).

Come sempre, il codice in questo esempio usa le API `tf.keras`, di cui potete imparare di più nella [guida Keras](https://www.tensorflow.org/guide/keras) di TensorFlow.

In entrambi gli esempi precedenti—[classificazione di testo](https://www.tensorflow.org/tutorials/keras/text_classification_with_hub) e [previsione di consumo carburante](https://www.tensorflow.org/tutorials/keras/regression) — abbiamo visto come l'accuratezza del nostro modello sui dati di validazione raggiunga il picco dopo l'addestramento per un certo numero di epoche, e poi ristagni o cominci a decrescere.

In altre parole, come il nostro modello si *sovradatti* (*overfit* n.d.t.) con i dati addestramento. Imparare come gestire il sovradattamento è importante. Sebbene sia spesso possibile raggiungere un'accuratezza alta sul *insieme di addestramento*, ciò che realmente vogliamo è sviluppare un modello che generalizzi bene sul *insieme di test* (o su dati che non ha mai visto prima).

L'opposto del sovradattamento è il *sottoadattamento*. Il sottoadattamento si verifica quando c'è ancora spazio di miglioramento sui dati di test. Ciò può accadere per un certo numero di ragioni: Se il modello non è abbastanza potente, se è sovra-regolarizzato, o, semplicemente, non è stato addestrato abbastanza. Ciò significa che la rete non ha appreso i pattern rilevanti nei dati di addestramento.

Se lo allenate per un tempo troppo lungo, il modello inizierà a sovradattarsi ed ad imparare pattern dai dati di addestramento che non si generalizzeranno ai dati di test. Dobbiamo trovare un compromesso. Capire come addestrare per un numero appropriato di epoche, come vedremo nel seguito, è una competenza utile.

La soluzione migliore per prevenire il sovradattaemnto è usare dati di addestramento più completi. Il dataset dovrebbe coprire lo spettro completo degli input che il modello possa aspettarsi di trattare. Ulteriori dati possono essere utili solo se coprono nuovi ed interessanti casi.

Un modello addestrato su un insieme più completo di dati generalizzerà naturalmente meglio. Quando ciò non fosse possibile, la soluzione migliore è usare tecniche come la regolarizzazione. Esse impongono dei vincoli alla quantità ed al tipo di informazioni che il vostro modello può immagazzinare. Se una rete può memorizzare solo un piccolo numero di pattern, il processo di ottimizzazione lo costringerà a focalizzarsi sui patern più promettenti, che hanno la migliore probabilità di generalizzare bene.

In questo notebook, esploreremo diverse tecniche di regolarizzazione, e le useremo per migliorare un modello di classificazione.

## Setup

Prima di iniziare,importiamo i pacchetti necessari:

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import regularizers

print(tf.__version__)

In [0]:
!pip install git+https://github.com/tensorflow/docs

import tensorflow_docs as tfdocs
import tensorflow_docs.modeling
import tensorflow_docs.plots

In [0]:
from  IPython import display
from matplotlib import pyplot as plt

import numpy as np

import pathlib
import shutil
import tempfile


In [0]:
logdir = pathlib.Path(tempfile.mkdtemp())/"tensorboard_logs"
shutil.rmtree(logdir, ignore_errors=True)


## Il Dataset Higgs

L'obiettivo di questo tutorial non è fare fisica delle particelle, perciò non approfondiamo i dettagli del dataset. Esso contiene  11&#x202F;000&#x202F;000 esempi, con 28 caratteristiche ciascuno, ed una etichetta di classificazione binaria.

In [0]:
gz = tf.keras.utils.get_file('HIGGS.csv.gz', 'https://archive.ics.uci.edu/ml/machine-learning-databases/00280/HIGGS.csv.gz')

In [0]:
FEATURES = 28

La classe `tf.data.experimental.CsvDataset` può essere usata per leggere record csv direttamente da un file gzip senza passi intermedi di decompressione.

In [0]:
ds = tf.data.experimental.CsvDataset(gz,[float(),]*(FEATURES+1), compression_type="GZIP")

La classe che legge il cvs restituisce una lista di scalari per ogni record. La funzione seguente re-impacchetta tale lista in una coppia (vettore_di_caratteristiche, etichetta).

In [0]:
def pack_row(*row):
  label = row[0]
  features = tf.stack(row[1:],1)
  return features, label

TensorFlow è più efficiente quando tratta grandi lotti di dati.

Così, invece di re-impacchettare ciascuna riga individualmente, costruiamo un nuovo `Dataset` che contiene lotti di 10000-esempi, applichiamo la funzione `pack_row` a ciascun lotto, e quindi separiamo i lotti in singoli record:

In [0]:
packed_ds = ds.batch(10000).map(pack_row).unbatch()

Diamo uno sguardo ad alcuni dei record di questo nuovo `packed_ds`.

Le caratteristiche non sono perfettamente normalizzate, ma questo non è un problema per questo tutorial.

In [0]:
for features,label in packed_ds.batch(1000).take(1):
  print(features[0])
  plt.hist(features.numpy().flatten(), bins = 101)

Per mantenere questo tutorial relativamente breve, usiamo solo i primi 1000 campioni per la validazione, ed i successivi 10000 per l'addestramento:

In [0]:
N_VALIDATION = int(1e3)
N_TRAIN = int(1e4)
BUFFER_SIZE = int(1e4)
BATCH_SIZE = 500
STEPS_PER_EPOCH = N_TRAIN//BATCH_SIZE

I metodi `Dataset.skip` e `Dataset.take` rendono semplice la cosa.

Allo stesso tempo, usare il metodo `Dataset.cache` assicura che il caricatore non abbia bisogno di ri-leggerei dati dal file ad ogni nuova epoca:

In [0]:
validate_ds = packed_ds.take(N_VALIDATION).cache()
train_ds = packed_ds.skip(N_VALIDATION).take(N_TRAIN).cache()

In [0]:
train_ds

Questi dataset restituiscono esempi singoli. Usiamo il metodo `.batch` per creare lotti di dimensione appropriata per l'addestramento. Prima della confezione dei lotti ricordiamo anche di `.shuffle` (*mescolare* n.d.t.) e `.repeat` (*ripetere* n.d.t.) l'insieme di addestramento.

In [0]:
validate_ds = validate_ds.batch(BATCH_SIZE)
train_ds = train_ds.shuffle(BUFFER_SIZE).repeat().batch(BATCH_SIZE)

## Dimostrare il sovradattamento

La via più semplice per prevenire il sovradattamento è cominciare con un modello piccolo: un modello con un piccolo numero di parametri da apprendere (determinati dal numero di livelli e dal numero di unità per livello). Nel deep learning, spesso si fa riferimento al numero di parametri che possono essere appresi da un modello, come la "capacity" del modello.

Intuitivamente, un modello con più parametri avrà maggiore "capacità di memorizzazione" e quindi sarà capace di imparare facilmente una corrispondenza perfetta simil-vocabolario tra i campioni di addestramento ed i relativi obiettivi, una mappatura senza nessun potere di generalizzazione, ma ciò sarebbe inutile per fare previsioni su dati mai visti.

Teniamo sempre a mente: i modelli di deep learning tendono ad essere bravi nell'aderire ai dati di addestramneto, ma la sfida reale è la generalizzazione, non l'adattamento.

d'altra parte, se la rete ha limitate risorse di memorizzazione, non sarà capace di imparare la mappatura così facilmente. Per minimizzare la sua perdita, dovrà apprendere rappresentazioni compresse che hanno maggiore potere predittivo. Allo stesso tempo, se fate il vostro modello troppo piccolo, esso ha difficoltà ad aderire ai dati di addestramento. C'è da fare un bilancio tra "capacità eccessiva" e "capacità insufficiente".

Sfortunatamente, non c'è nessuna formula magica per stabilire la dimensione giusta o l'architettura del vostro modello (in altre parole, il numero dei livelli , o la dimensione giusta di ciascun livello). Dovrete fare esperimenti usando architetture diverse.

Per trovare la dimensione giusta del modello, è meglio partire con relativamente pochi livelli e parametri, quindi cominciare ad aumentare la dimensione dei livelli o ad aggiungere nuovi livelli fino a che non vedete diminuire i ritorni sulla perdita di validazione.

Come punto di partenza, iniziate con un modello semplice che usa solo `layers.Dense`, poi create versioni più grandi e confrontatele.

### Procedura di addestramento

Molti modelli si addestrano meglio se riducete gradualmente il tasso di apprendimento durante l'addestramento. Per ridurre il tasso di apprendimento nel tempo, usiamo `optimizers.schedules`:

In [0]:
lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
  0.001,
  decay_steps=STEPS_PER_EPOCH*1000,
  decay_rate=1,
  staircase=False)

def get_optimizer():
  return tf.keras.optimizers.Adam(lr_schedule)

Il codice di cui sopra usa `schedules.InverseTimeDecay` per diminuire iperbolicamente il tasso di apprendimento ad 1/2 del tasso base dopo 1000 epoche, 1/3 a 2000 epoche e così via.

In [0]:
step = np.linspace(0,100000)
lr = lr_schedule(step)
plt.figure(figsize = (8,6))
plt.plot(step/STEPS_PER_EPOCH, lr)
plt.ylim([0,max(plt.ylim())])
plt.xlabel('Epoch')
_ = plt.ylabel('Learning Rate')


Ogni modello in questo tutorial userà la medesima configurazione di addestramento. Perciò la impostiamo in modo riusabile, partendo dalla lista delle callback.

L'addestramento di questo tutorial gira per molte epoche brevi. Per ridurre lo sforzo di tracciamento usiamo `tfdocs.EpochDots` che utilizza un semplice `.` per ogni epoca e, un insieme completo di metriche ogni 100 epoche.

In seguito, includiamo `callbacks.EarlyStopping` per evitare lunghi tempi di addestramento non necessari. notate che questa callback è impostata per monitorare la `val_binary_crossentropy`, non la `val_loss`. Questa differenza sarà importante in seguito.

Usiamo `callbacks.TensorBoard` per generare i tracciati TensorBoard per l'addestramento.



In [0]:
def get_callbacks(name):
  return [
    tfdocs.modeling.EpochDots(),
    tf.keras.callbacks.EarlyStopping(monitor='val_binary_crossentropy', patience=200),
    tf.keras.callbacks.TensorBoard(logdir/name),
  ]

Analogamente ogni modello userà le stesse impostazioni `Model.compile` e `Model.fit` settings:

In [0]:
def compile_and_fit(model, name, optimizer=None, max_epochs=10000):
  if optimizer is None:
    optimizer = get_optimizer()
  model.compile(optimizer=optimizer,
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=[
                  tf.keras.losses.BinaryCrossentropy(
                      from_logits=True, name='binary_crossentropy'),
                  'accuracy'])

  model.summary()

  history = model.fit(
    train_ds,
    steps_per_epoch = STEPS_PER_EPOCH,
    epochs=max_epochs,
    validation_data=validate_ds,
    callbacks=get_callbacks(name),
    verbose=0)
  return history

### Tiny model

Start by training a linear model:

In [0]:
tiny_model = tf.keras.Sequential([
    layers.Dense(16, activation='elu', input_shape=(FEATURES,)),
    layers.Dense(1)
])

In [0]:
size_histories = {}

In [0]:
size_histories['Tiny'] = compile_and_fit(tiny_model, 'sizes/Tiny')

Now check how the model did:

In [0]:
plotter = tfdocs.plots.HistoryPlotter(metric = 'binary_crossentropy', smoothing_std=10)
plotter.plot(size_histories)
plt.ylim([0.5, 0.7])

### Small model

To see if you can beat the performance of the small model, progressively train some larger models.

Try two hidden layers with 16 units each:

In [0]:
small_model = tf.keras.Sequential([
    # `input_shape` is only required here so that `.summary` works.
    layers.Dense(16, activation='elu', input_shape=(FEATURES,)),
    layers.Dense(16, activation='elu'),
    layers.Dense(1)
])

In [0]:
size_histories['Small'] = compile_and_fit(small_model, 'sizes/Small')

### Medium model

Now try 3 hidden layers with 64 units each:

In [0]:
medium_model = tf.keras.Sequential([
    layers.Dense(64, activation='elu', input_shape=(FEATURES,)),
    layers.Dense(64, activation='elu'),
    layers.Dense(64, activation='elu'),
    layers.Dense(1)
])

And train the model using the same data:

In [0]:
size_histories['Medium']  = compile_and_fit(medium_model, "sizes/Medium")

### Large model

As an exercise, you can create an even larger model, and see how quickly it begins overfitting.  Next, let's add to this benchmark a network that has much more capacity, far more than the problem would warrant:

In [0]:
large_model = tf.keras.Sequential([
    layers.Dense(512, activation='elu', input_shape=(FEATURES,)),
    layers.Dense(512, activation='elu'),
    layers.Dense(512, activation='elu'),
    layers.Dense(512, activation='elu'),
    layers.Dense(1)
])

And, again, train the model using the same data:

In [0]:
size_histories['large'] = compile_and_fit(large_model, "sizes/large")

### Plot the training and validation losses

The solid lines show the training loss, and the dashed lines show the validation loss (remember: a lower validation loss indicates a better model).

While building a larger model gives it more power, if this power is not constrained somehow it can easily overfit to the training set.

In this example, typically, only the `"Tiny"` model manages to avoid overfitting altogether, and each of the larger models overfit the data more quickly. This becomes so severe for the `"large"` model that you need to switch the plot to a log-scale to really see what's happening.

This is apparent if you plot and compare the validation metrics to the training metrics.

* It's normal for there to be a small difference.
* If both metrics are moving in the same direction, everything is fine.
* If the validation metric begins to stagnate while the training metric continues to improve, you are probably close to overfitting.
* If the validation metric is going in the wrong direction, the model is clearly overfitting.

In [0]:
plotter.plot(size_histories)
a = plt.xscale('log')
plt.xlim([5, max(plt.xlim())])
plt.ylim([0.5, 0.7])
plt.xlabel("Epochs [Log Scale]")

Note: All the above training runs used the `callbacks.EarlyStopping` to end the training once it was clear the model was not making progress.

### View in TensorBoard

These models all wrote TensorBoard logs during training.

To open an embedded  TensorBoard viewer inside a notebook, copy the following into a code-cell:

```
%tensorboard --logdir {logdir}/sizes
```

You can view the [results of a previous run](https://tensorboard.dev/experiment/vW7jmmF9TmKmy3rbheMQpw/#scalars&_smoothingWeight=0.97) of this notebook on [TensorBoard.dev](https://tensorboard.dev/).

TensorBoard.dev is a managed experience for hosting, tracking, and sharing ML experiments with everyone.

It's also included in an `<iframe>` for convenience:

In [0]:
display.IFrame(
    src="https://tensorboard.dev/experiment/vW7jmmF9TmKmy3rbheMQpw/#scalars&_smoothingWeight=0.97",
    width="100%", height="800px")

If you want to share TensorBoard results you can upload the logs to [TensorBoard.dev](https://tensorboard.dev/) by copying the following into a code-cell.

Note: This step requires a Google account.

```
!tensorboard dev upload --logdir  {logdir}/sizes
```

Caution: This command does not terminate. It's designed to continuously upload the results of long-running experiments. Once your data is uploaded you need to stop it using the "interrupt execution" option in your notebook tool.

## Strategies to prevent overfitting

Before getting into the content of this section copy the training logs from the `"Tiny"` model above, to use as a baseline for comparison.

In [0]:
shutil.rmtree(logdir/'regularizers/Tiny', ignore_errors=True)
shutil.copytree(logdir/'sizes/Tiny', logdir/'regularizers/Tiny')

In [0]:
regularizer_histories = {}
regularizer_histories['Tiny'] = size_histories['Tiny']

### Add weight regularization



You may be familiar with Occam's Razor principle: given two explanations for something, the explanation most likely to be correct is the "simplest" one, the one that makes the least amount of assumptions. This also applies to the models learned by neural networks: given some training data and a network architecture, there are multiple sets of weights values (multiple models) that could explain the data, and simpler models are less likely to overfit than complex ones.

A "simple model" in this context is a model where the distribution of parameter values has less entropy (or a model with fewer parameters altogether, as we saw in the section above). Thus a common way to mitigate overfitting is to put constraints on the complexity of a network by forcing its weights only to take small values, which makes the distribution of weight values more "regular". This is called "weight regularization", and it is done by adding to the loss function of the network a cost associated with having large weights. This cost comes in two flavors:

* [L1 regularization](https://developers.google.com/machine-learning/glossary/#L1_regularization), where the cost added is proportional to the absolute value of the weights coefficients (i.e. to what is called the "L1 norm" of the weights).

* [L2 regularization](https://developers.google.com/machine-learning/glossary/#L2_regularization), where the cost added is proportional to the square of the value of the weights coefficients (i.e. to what is called the squared "L2 norm" of the weights). L2 regularization is also called weight decay in the context of neural networks. Don't let the different name confuse you: weight decay is mathematically the exact same as L2 regularization.

L1 regularization pushes weights towards exactly zero encouraging a sparse model. L2 regularization will penalize the weights parameters without making them sparse since the penalty goes to zero for small weights. one reason why L2 is more common.

In `tf.keras`, weight regularization is added by passing weight regularizer instances to layers as keyword arguments. Let's add L2 weight regularization now.

In [0]:
l2_model = tf.keras.Sequential([
    layers.Dense(512, activation='elu',
                 kernel_regularizer=regularizers.l2(0.001),
                 input_shape=(FEATURES,)),
    layers.Dense(512, activation='elu',
                 kernel_regularizer=regularizers.l2(0.001)),
    layers.Dense(512, activation='elu',
                 kernel_regularizer=regularizers.l2(0.001)),
    layers.Dense(512, activation='elu',
                 kernel_regularizer=regularizers.l2(0.001)),
    layers.Dense(1)
])

regularizer_histories['l2'] = compile_and_fit(l2_model, "regularizers/l2")

`l2(0.001)` means that every coefficient in the weight matrix of the layer will add `0.001 * weight_coefficient_value**2` to the total **loss** of the network.

That is why we're monitoring the `binary_crossentropy` directly. Because it doesn't have this regularization component mixed in.

So, that same `"Large"` model with an `L2` regularization penalty performs much better:



In [0]:
plotter.plot(regularizer_histories)
plt.ylim([0.5, 0.7])

As you can see, the `"L2"` regularized model is now much more competitive with the the `"Tiny"` model. This `"L2"` model is also much more resistant to overfitting than the `"Large"` model it was based on despite having the same number of parameters.

#### More info

There are two important things to note about this sort of regularization.

**First:** if you are writing your own training loop, then you need to be sure to ask the model for it's regularization losses.

In [0]:
result = l2_model(features)
regularization_loss=tf.add_n(l2_model.losses)

**Second:** This implementation works by adding the weight penalties to the model's loss, and then applying a standard optimization procedure after that.

There is a second approach that instead only runs the optimizer on the raw loss, and then while applying the calculated step the optimizer also applies some weight decay. This "Decoupled Weight Decay" Is seen in optimizers like `optimizers.FTRL` and `optimizers.AdamW`.

### Add dropout

Dropout is one of the most effective and most commonly used regularization techniques for neural networks, developed by Hinton and his students at the University of Toronto.

The intuitive explanation for dropout is that because individual nodes in the network cannot rely on the output of the others, each node must output features that are useful on their own.

Dropout, applied to a layer, consists of randomly "dropping out" (i.e. set to zero) a number of output features of the layer during training. Let's say a given layer would normally have returned a vector [0.2, 0.5, 1.3, 0.8, 1.1] for a given input sample during training; after applying dropout, this vector will have a few zero entries distributed at random, e.g. [0, 0.5,
1.3, 0, 1.1].

The "dropout rate" is the fraction of the features that are being zeroed-out; it is usually set between 0.2 and 0.5. At test time, no units are dropped out, and instead the layer's output values are scaled down by a factor equal to the dropout rate, so as to balance for the fact that more units are active than at training time.

In `tf.keras` you can introduce dropout in a network via the Dropout layer, which gets applied to the output of layer right before.

Let's add two Dropout layers in our network to see how well they do at reducing overfitting:

In [0]:
dropout_model = tf.keras.Sequential([
    layers.Dense(512, activation='elu', input_shape=(FEATURES,)),
    layers.Dropout(0.5),
    layers.Dense(512, activation='elu'),
    layers.Dropout(0.5),
    layers.Dense(512, activation='elu'),
    layers.Dropout(0.5),
    layers.Dense(512, activation='elu'),
    layers.Dropout(0.5),
    layers.Dense(1)
])

regularizer_histories['dropout'] = compile_and_fit(dropout_model, "regularizers/dropout")

In [0]:
plotter.plot(regularizer_histories)
plt.ylim([0.5, 0.7])

It's clear from this plot that both of these regularization approaches improve the behavior of the `"Large"` model. But this still doesn't beat even the `"Tiny"` baseline.

Next try them both, together, and see if that does better.

### Combined L2 + dropout

In [0]:
combined_model = tf.keras.Sequential([
    layers.Dense(512, kernel_regularizer=regularizers.l2(0.0001),
                 activation='elu', input_shape=(FEATURES,)),
    layers.Dropout(0.5),
    layers.Dense(512, kernel_regularizer=regularizers.l2(0.0001),
                 activation='elu'),
    layers.Dropout(0.5),
    layers.Dense(512, kernel_regularizer=regularizers.l2(0.0001),
                 activation='elu'),
    layers.Dropout(0.5),
    layers.Dense(512, kernel_regularizer=regularizers.l2(0.0001),
                 activation='elu'),
    layers.Dropout(0.5),
    layers.Dense(1)
])

regularizer_histories['combined'] = compile_and_fit(combined_model, "regularizers/combined")

In [0]:
plotter.plot(regularizer_histories)
plt.ylim([0.5, 0.7])

This model with the `"Combined"` regularization is obviously the best one so far.

### View in TensorBoard

These models also recorded TensorBoard logs.

To open an embedded  tensorboard viewer inside a notebook, copy the following into a code-cell:

```
%tensorboard --logdir {logdir}/regularizers
```

You can view the [results of a previous run](https://tensorboard.dev/experiment/fGInKDo8TXes1z7HQku9mw/#scalars&_smoothingWeight=0.97) of this notebook on [TensorDoard.dev](https://tensorboard.dev/).

It's also included in an `<iframe>` for convenience:

In [0]:
display.IFrame(
    src="https://tensorboard.dev/experiment/fGInKDo8TXes1z7HQku9mw/#scalars&_smoothingWeight=0.97",
    width = "100%",
    height="800px")


This was uploaded with:

```
!tensorboard dev upload --logdir  {logdir}/regularizers
```

## Conclusions

To recap: here are the most common ways to prevent overfitting in neural networks:

* Get more training data.
* Reduce the capacity of the network.
* Add weight regularization.
* Add dropout.

Two important approaches not covered in this guide are:

* data-augmentation
* batch normalization

Remember that each method can help on its own, but often combining them can be even more effective.